In [1]:
# @title Installing libraries
!pip install difflib
!pip install sentence-transformers
!pip install scikit-learn
!pip install spacy
!pip install numpy
!pip install sentence-transformers
!pip install string2string
!pip install textstat
# After installing spacy, you also need to download a language model
# python -m spacy download en_core_web_sm

ERROR: Could not find a version that satisfies the requirement difflib (from versions: none)
ERROR: No matching distribution found for difflib
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.5-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.4 MB/s eta 0:00:00
Using cached pybind11

In [3]:
# @title Importing Libraries and Models
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import spacy
import numpy as np
import spacy
import difflib
import textstat
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sklearn.metrics.pairwise import cosine_similarity
import torch
# from string2string.misc import ModelEmbeddings
from spacy import displacy
nli_model = pipeline("text-classification", model="facebook/bart-large-mnli")
model_transformer = SentenceTransformer('all-MiniLM-L6-v2')
# nlp_sm = spacy.load("en_core_web_lg")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [37]:
# @title NLI (example)

from transformers import pipeline

nli_model = pipeline("text-classification", model="facebook/bart-large-mnli")

context = "Ruskin Bond is a famous Indian author known for his contributions to literature."
statement = "Ruskin Bond is an football"
premise_hypothesis_pair = f"{context} {statement}"
result = nli_model(premise_hypothesis_pair)
print(result)
print(result[0]['score'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'contradiction', 'score': 0.8150558471679688}]


**Natural Language Inference**

This method allows the model to determine whether a statement (hypothesis) logically follows from the context (premise).

# FINAL FUNCTIONS

In [51]:
def compare_texts(text1, text2):
    # print("1. Text Comparison:")
    d = difflib.Differ()
    diff = list(d.compare(text1.splitlines(), text2.splitlines()))

    additions = [line[2:] for line in diff if line.startswith('+ ')]
    deletions = [line[2:] for line in diff if line.startswith('- ')]

    similarity_ratio = difflib.SequenceMatcher(None, text1, text2).ratio()
    return similarity_ratio

def semantic_similarity(text1, text2,model):
    # print("\n2. Semantic Similarity:")
    embedding1 = model.encode(text1)
    embedding2 = model.encode(text2)
    similarity = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    return similarity

def readability_and_style(text2):
    # print("\n3. Readability and Style Comparison:")
    readability2 = textstat.flesch_reading_ease(text2)
    return readability2/100

def factuality(context,statement):
    premise_hypothesis_pair = f"{context} {statement}"
    result = nli_model(premise_hypothesis_pair)
    # print(result[0]['label'])
    if(result[0]['label']=="entailment"):
       return (1-result[0]['score'])/2
    if(result[0]['label']=="neutral"):
       return result[0]['score']/2
    else:
        return result[0]['score']

In [52]:
def evalutaion_score(text1,text2):
    s1 = compare_texts(text1,text2)
    s2 = semantic_similarity(text1,text2,model_transformer)
    s3 = readability_and_style(text2)
    fct = factuality(text1,text2)
    print(s1,s2,s3,fct)

    final_score = fct*fct*((s1+s2+s3)/3)
    return final_score

# EXAMPLES

In [56]:
# RESPONSES GENERATED BY SUPERLLM AND OUR LLM FOR THE QUESTION What is Percy Jakson family background?

text1 = "Percy Jackson is the son of Sally Jackson and Gabe Jackson."
text2 = "Percy Jackson was born to a Hong Kong-based Cantonese family."

eval_score = evalutaion_score(text1,text2)
print("final evaluation score = ",eval_score)

0.4 0.5914879 0.6133 0.9985958933830261
final evaluation score =  0.5334281539368357


In [57]:
# RESPONSES GENERATED BY SUPERLLM AND OUR LLM FOR THE QUESTION What does Saina Nehwal do?

text1 = "Saina Nehwal was a prominent Indian poet, writer, and independence activist."
text2 = "Saina Nehwal is a professional badminton player from India."

eval_score = evalutaion_score(text1,text2)
print("final evaluation score = ",eval_score)

0.45925925925925926 0.75848377 0.3696 0.9915459752082825
final evaluation score =  0.5202058668072767


In [58]:
# SAMPLE SIMILAR TEXT (NOT GENERATED BY OUR LLM)
text1 = "Saina Nehwal is a great Indian Badminton player"
text2 = "Saina Nehwal is a professional badminton player from India."

eval_score = evalutaion_score(text1,text2)
print("final evaluation score = ",eval_score)

0.6981132075471698 0.9503701 0.3696 0.028180986642837524
final evaluation score =  0.0005342323931579049


# CONCLUSION

1. **When our model successfully alters the information of the RAW agent, we achieve a score of ≥0.35**.

2. **It is important to note that a score of 0.4 is sufficient to conclude that our model performs well, given that due to certain calculation constraints, the model cannot achieve a score close to 1.**

3. **When our model fails to alter the information of the RAW agent, we obtain a score of ≤0.15.**